# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import FreqDist

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s = re.sub(r'(http://\w+.\w+)', '', s)
    s = re.sub(r'[^A-Za-z0-9]+', ' ', s)
    s = re.sub(r'(\d+)', '', s)
    s = re.sub(r'  ', ' ', s)
    s = s.strip()
    s = s.lower()
    return s

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    stemmed_list = []
    limmatized_list = []
    for i in l:
        stemmer = PorterStemmer().stem(i)
        stemmed_list.append(i)
    for a in stemmed_list:
        lemmatizer = WordNetLemmatizer().lemmatize(a)
        limmatized_list.append(a)
    
    return limmatized_list


def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    new_list = []
    stop_words = list(stopwords.words('english'))
    for i in l:
        if i not in stop_words:
            new_list.append(i)
    return new_list

In [3]:
sentiment = pd.read_csv('..//data//Sentiment140.csv//Sentiment140.csv')

In [4]:
#sentiment_sample = sentiment.loc[:20000].copy()
sentiment_sample = sentiment.sample(n=20000)
sentiment_sample.head()

,target,id,date,flag,user,text
1565265,4,2187504543,Mon Jun 15 20:22:49 PDT 2009,NO_QUERY,Dannabug,"Ok, seriously, my twitter binge is over now. I..."
724042,0,2261957552,Sat Jun 20 21:38:19 PDT 2009,NO_QUERY,a_n_t_y,@Pascalia huhu I wish my pacar always in sydne...
603868,0,2221777949,Thu Jun 18 05:59:24 PDT 2009,NO_QUERY,TREND_D,Missed my train.......
730570,0,2263605545,Sun Jun 21 01:16:48 PDT 2009,NO_QUERY,leesarhh,Is not ready for the Methods SAC tmr
1127941,4,1975279133,Sat May 30 14:49:31 PDT 2009,NO_QUERY,bk4theos,@LindelAnderson Can't wait 2 have her in class.


In [5]:
sentiment_sample.replace({'target': {4: 1}}, inplace = True)
sentiment_sample['target'] = sentiment_sample['target'].astype('bool')

sentiment_sample.head()

,target,id,date,flag,user,text
1565265,True,2187504543,Mon Jun 15 20:22:49 PDT 2009,NO_QUERY,Dannabug,"Ok, seriously, my twitter binge is over now. I..."
724042,False,2261957552,Sat Jun 20 21:38:19 PDT 2009,NO_QUERY,a_n_t_y,@Pascalia huhu I wish my pacar always in sydne...
603868,False,2221777949,Thu Jun 18 05:59:24 PDT 2009,NO_QUERY,TREND_D,Missed my train.......
730570,False,2263605545,Sun Jun 21 01:16:48 PDT 2009,NO_QUERY,leesarhh,Is not ready for the Methods SAC tmr
1127941,True,1975279133,Sat May 30 14:49:31 PDT 2009,NO_QUERY,bk4theos,@LindelAnderson Can't wait 2 have her in class.


In [6]:
sentiment_sample.target.value_counts()

False    10070
True      9930
Name: target, dtype: int64

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](../images/data-cleaning-results.png)

In [7]:
sentiment_sample['text_processed'] = sentiment_sample['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)

In [8]:
sentiment_sample.head()

,target,id,date,flag,user,text,text_processed
1565265,True,2187504543,Mon Jun 15 20:22:49 PDT 2009,NO_QUERY,Dannabug,"Ok, seriously, my twitter binge is over now. I...","[ok, seriously, twitter, binge, putting, lap, ..."
724042,False,2261957552,Sat Jun 20 21:38:19 PDT 2009,NO_QUERY,a_n_t_y,@Pascalia huhu I wish my pacar always in sydne...,"[pascalia, huhu, wish, pacar, always, sydney]"
603868,False,2221777949,Thu Jun 18 05:59:24 PDT 2009,NO_QUERY,TREND_D,Missed my train.......,"[missed, train]"
730570,False,2263605545,Sun Jun 21 01:16:48 PDT 2009,NO_QUERY,leesarhh,Is not ready for the Methods SAC tmr,"[ready, methods, sac, tmr]"
1127941,True,1975279133,Sat May 30 14:49:31 PDT 2009,NO_QUERY,bk4theos,@LindelAnderson Can't wait 2 have her in class.,"[lindelanderson, wait, class]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
column_list = []
for i in sentiment_sample['text_processed']:
    for l in i:
        column_list.append(l)

len(column_list)

153246

In [10]:
import itertools

word_dict = FreqDist(column_list)

Sorted_word_freq = pd.DataFrame(list(word_dict.items()), columns = ["Word","Frequency"]).sort_values(by = 'Frequency', ascending = False)
Sorted_word_freq_top_5000 = Sorted_word_freq.iloc[:5000]
Sorted_word_freq_top_5000

,Word,Frequency
131,good,1169
296,day,1090
261,like,968
52,get,963
109,got,917
418,go,900
53,quot,882
172,today,853
365,work,848
55,going,835


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [11]:
list_of_top_words = list(Sorted_word_freq_top_5000['Word'])

In [12]:
sentiment_sample.shape[0]

20000

In [13]:
sentiment_sample['text_processed'].iloc[100]

['hard', 'move', 'need', 'learn', 'let', 'go']

In [14]:
def find_features(document):
    '''
    Input all words, returns list with tuples.
    Each tuple contains a dict with True's and False's
    for each word and the label y = {'neg', 'pos'}
    '''
    words = set(list_of_top_words)
    features = {}
    for w in document:
        features[w] = True if w in words else False

    return features

#print(find_features(sentiment_sample['text_processed']))
#Creat the feature set consisting of instances with their categories and words

#featuresets = [(find_features(sentiment_sample['text_processed']), sentiment_sample['target']) for row in sentiment_sample.iterrows()]


featuresets = []

for i in range(sentiment_sample.shape[0]):
    A = (find_features(sentiment_sample['text_processed'].iloc[i]), sentiment_sample['target'].iloc[i])
    featuresets.append(A)

In [15]:
len(featuresets)

20000

In [16]:
featuresets[0]

({'ok': True,
  'seriously': True,
  'twitter': True,
  'binge': False,
  'putting': True,
  'lap': True,
  'top': True,
  'bed': True},
 True)

[![Building Features](../images/building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [17]:
training_set = featuresets[:18000]
testing_set = featuresets[18000:20000]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](../images/nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [18]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [19]:
print(nltk.classify.accuracy(classifier, training_set))

0.922


In [20]:
classifier.show_most_informative_features(15)

Most Informative Features
            followfriday = True             True : False  =     23.3 : 1.0
               wonderful = True             True : False  =     22.6 : 1.0
                   tummy = True            False : True   =     21.4 : 1.0
                     sad = True            False : True   =     20.0 : 1.0
                    ouch = True            False : True   =     15.5 : 1.0
                    sick = True            False : True   =     14.9 : 1.0
                   hates = True            False : True   =     14.8 : 1.0
                     vip = True             True : False  =     14.5 : 1.0
                     www = True             True : False  =     14.3 : 1.0
                   proud = True             True : False  =     11.8 : 1.0
                fabulous = True             True : False  =     11.8 : 1.0
                pleasure = True             True : False  =     11.8 : 1.0
                   sadly = True            False : True   =     11.7 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [26]:
print(nltk.classify.accuracy(classifier1, testing_set))

0.7115


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [23]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [24]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [25]:
# your code here